<a href="https://colab.research.google.com/github/NeeteshDadwariya/CS6375/blob/main/assignments/assignment2/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

In [34]:
class NeuralNetwork:

  def __init__(self, data_url):
    self.df = pd.read_csv(data_url)
    self.stdscalar = None

    display(self.df.head(10))
    display(self.df.describe())

  def pre_process(self):
    #Check for null values
    print("\nNull Values:")
    display(self.df.isnull().sum())

    #Check for NA values
    print("\nNA Values:")
    display(self.df.isna().sum())

    #Check for duplicates values
    print("\nDuplicate Values:")
    display(self.df.duplicated().sum())

    return self.df

  #Split the target variable from the data
  def split_X_and_y(self):
    no_rows, no_cols = self.df.shape[0], self.df.shape[1]
    X =  self.df.iloc[:, 0:(no_cols -1)]
    y = self.df.iloc[:, no_cols-1]
    return X,y

  #Normalizing the data 
  def scale_data(self, df, is_train: bool):
    if is_train:
      stdscalar = StandardScalar()
      stdscalar.fit(df)
    return stdscalar.transform(df)
    

    


In [35]:
network = NeuralNetwork("https://archive.org/download/heart_data/heart_data.csv")

,age,sex,chest_pain_type,resting_blood_pressure,serum_cholestrol_mg_dl,fasting_blood_sugar_mg_dl,resting_electrocardiographic_results,max_heart_rate_achieved,exercise_induced_angina,oldpeak,slope_st_segment,major_vessel_count,thal,heart_disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,1
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,0
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,1
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,0
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,0
5,65,1,4,120,177,0,0,140,0,0.4,1,0,7,0
6,56,1,3,130,256,1,2,142,1,0.6,2,1,6,1
7,59,1,4,110,239,0,2,142,1,1.2,2,1,7,1
8,60,1,4,140,293,0,2,170,0,1.2,2,2,7,1
9,63,0,4,150,407,0,2,154,0,4.0,2,3,7,1


,age,sex,chest_pain_type,resting_blood_pressure,serum_cholestrol_mg_dl,fasting_blood_sugar_mg_dl,resting_electrocardiographic_results,max_heart_rate_achieved,exercise_induced_angina,oldpeak,slope_st_segment,major_vessel_count,thal,heart_disease
count,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.000000,270.00000,270.000000,270.000000,270.000000,270.000000
mean,54.433333,0.677778,3.174074,131.344444,249.659259,0.148148,1.022222,149.677778,0.329630,1.05000,1.585185,0.670370,4.696296,0.444444
std,9.109067,0.468195,0.950090,17.861608,51.686237,0.355906,0.997891,23.165717,0.470952,1.14521,0.614390,0.943896,1.940659,0.497827
min,29.000000,0.000000,1.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.00000,1.000000,0.000000,3.000000,0.000000
25%,48.000000,0.000000,3.000000,120.000000,213.000000,0.000000,0.000000,133.000000,0.000000,0.00000,1.000000,0.000000,3.000000,0.000000
50%,55.000000,1.000000,3.000000,130.000000,245.000000,0.000000,2.000000,153.500000,0.000000,0.80000,2.000000,0.000000,3.000000,0.000000
75%,61.000000,1.000000,4.000000,140.000000,280.000000,0.000000,2.000000,166.000000,1.000000,1.60000,2.000000,1.000000,7.000000,1.000000
max,77.000000,1.000000,4.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.20000,3.000000,3.000000,7.000000,1.000000


In [36]:
df = network.pre_process()


Null Values:


age                                     0
sex                                     0
chest_pain_type                         0
resting_blood_pressure                  0
serum_cholestrol_mg_dl                  0
fasting_blood_sugar_mg_dl               0
resting_electrocardiographic_results    0
max_heart_rate_achieved                 0
exercise_induced_angina                 0
oldpeak                                 0
slope_st_segment                        0
major_vessel_count                      0
thal                                    0
heart_disease                           0
dtype: int64


NA Values:


age                                     0
sex                                     0
chest_pain_type                         0
resting_blood_pressure                  0
serum_cholestrol_mg_dl                  0
fasting_blood_sugar_mg_dl               0
resting_electrocardiographic_results    0
max_heart_rate_achieved                 0
exercise_induced_angina                 0
oldpeak                                 0
slope_st_segment                        0
major_vessel_count                      0
thal                                    0
heart_disease                           0
dtype: int64


Duplicate Values:


0